# Scrape from [transfermarkt](https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1)

In [1]:
import bs4
import sys
import requests
import re
import pandas as pd
import numpy as np

In [123]:
def get_team_links(season):
    # season = 2020
    df = pd.DataFrame(columns=['team_name', 'club', 'team_num', 'link', 'season'])
    user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
    url = f'https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1/plus/?saison_id={season}'
    response = requests.get(url, headers={'User-Agent': user_agent})
    content = response.content.decode('utf-8')
    soup = bs4.BeautifulSoup(content, 'html.parser')
    table = soup.find('table', attrs={'class':'items'}).find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        club = row.find('a').find('img')['alt']
        info = row.find('a')['href'].split('/')
        team_name = info[1]
        team_num = info[4]
        team_link = f"https://www.transfermarkt.com/{team_name}/kader/verein/{team_num}/saison_id/{season}/plus/1"
        df.loc[len(df.index)] = [team_name, club, team_num, team_link, season]
    return df


In [124]:
team_links = get_team_links(2020)

In [175]:
def get_player_bio(team_links_row):
    err = []
    df = pd.DataFrame(columns=['first_name', 'last_name', 'player', 'club', 'bday', 'join', 'value', 'season'])
    url = team_links_row['link']
    season = team_links_row['season']
    club = team_links_row['club']
    user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
    response = requests.get(url, headers={'User-Agent': user_agent})
    content = response.content.decode('utf-8')
    soup = bs4.BeautifulSoup(content, 'html.parser')
    table = soup.find('table', attrs={'class':'items'}).find('tbody')
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        info = [ele.text.strip() for ele in cols]
        if len(info) > 3:
            s = [idx for idx in range(len(info[3])) if info[3][idx].isupper()][-2]
            name = info[3][:s]
            try:
                first, *last_temp = name.split()
                last = " ".join(last_temp).strip('.').strip()
            except:
                err.append([club, name])
                continue
            title = info[4]
            bday = info[5].split('(')[0].strip()
            if len(bday) < 8:
                bday = None
            height = info[7]
            foot = info[8]
            join = info[9]
            value = info[12]
            ddf = {
                'first_name': first,
                'last_name': last,
                'player': ". ".join([first[0].upper(), last]),
                'club': club,
                'bday': bday,
                'join': join,
                'value': value,
                'season': season
            }
            df = df.append(ddf, ignore_index=True)
            
    df['bday'] = pd.to_datetime(df['bday'])
    df['join'] = pd.to_datetime(df['join'])
    now = pd.Timestamp('now')
    df['age'] = (now - df['bday']).astype('<m8[Y]')
    df['join_age'] = (df['join'] - df['bday']).astype('<m8[Y]')
    
    df = df.drop(columns=['bday', 'join'])
        
    return df

In [176]:
dfs = []
for i in range(len(team_links)):
    dfs.append(get_player_bio(team_links.loc[i]))

player_df = pd.concat(dfs, ignore_index=True)

,first_name,last_name,player,club,value,season,age,join_age
0,Kenneth,Vermeer,K. Vermeer,Los Angeles FC,€800Th.,2020,35.0,34.0
1,Pablo,Sisniega,P. Sisniega,Los Angeles FC,€500Th.,2020,25.0,23.0
2,Tomás,Romero,T. Romero,Los Angeles FC,€100Th.,2020,20.0,20.0
3,Eddie,Segura,E. Segura,Los Angeles FC,€2.50m,2020,24.0,22.0
4,Jesús,Murillo,J. Murillo,Los Angeles FC,€1.20m,2020,27.0,26.0
...,...,...,...,...,...,...,...,...
723,Tate,Schmitt,T. Schmitt,Real Salt Lake City,€200Th.,2020,23.0,21.0
724,Christopher,Garcia,C. Garcia,Real Salt Lake City,€100Th.,2020,18.0,17.0
725,Douglas,Martínez,D. Martínez,Real Salt Lake City,€600Th.,2020,23.0,22.0
726,Rubio,Rubin,R. Rubin,Real Salt Lake City,€400Th.,2020,25.0,24.0


In [177]:
player_df = pd.concat(dfs, ignore_index=True)

In [192]:
player_df.to_csv('data/player_bio.csv')

In [213]:
def get_one_page(season, page=None):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
    url = f'https://www.mlssoccer.com/stats/season?page={page}&franchise=select&year={season}'
    response = requests.get(url, headers={'User-Agent': user_agent})
    content = response.content.decode('utf-8')
    soup = bs4.BeautifulSoup(content, 'html.parser')
    table = soup.find('tbody')
    rows = table.find_all('tr')
    cols = rows[0].find_all('td')
    df = pd.DataFrame(columns=[ele['data-title'] for ele in cols])
    for row in rows:
        cols = row.find_all('td')
        df.loc[len(df.index)] = [ele.text.strip() for ele in cols]
    return df

def get_player_stats(season):
    last = 27 # last page number, manually input here
    dfs = [get_one_page(season, page=p) for p in range(last)]
    return pd.concat(dfs, ignore_index=True)

In [216]:
stat_df = get_player_stats(2020)

In [217]:
stat_df.to_csv("data/play_time.csv")

In [223]:
sorted(player_df['club'].unique())

['Atlanta United FC',
 'Austin FC',
 'Chicago Fire FC',
 'Club de Foot Montréal',
 'Colorado Rapids',
 'Columbus Crew SC',
 'D.C. United',
 'FC Cincinnati',
 'FC Dallas',
 'Houston Dynamo FC',
 'Inter Miami CF',
 'Los Angeles FC',
 'Los Angeles Galaxy',
 'Minnesota United FC',
 'Nashville SC',
 'New England Revolution',
 'New York City FC',
 'New York Red Bulls',
 'Orlando City SC',
 'Philadelphia Union',
 'Portland Timbers',
 'Real Salt Lake City',
 'San Jose Earthquakes',
 'Seattle Sounders FC',
 'Sporting Kansas City',
 'Toronto FC',
 'Vancouver Whitecaps FC']

In [224]:
sorted(stat_df['Club'].unique())

['ATL',
 'ATX',
 'CHI',
 'CIN',
 'CLB',
 'CLT',
 'COL',
 'DAL',
 'DC',
 'HOU',
 'LA',
 'LAFC',
 'LFC',
 'MCF',
 'MIA',
 'MIN',
 'MTL',
 'NE',
 'NSH',
 'NY',
 'NYC',
 'ORL',
 'PHI',
 'POR',
 'RBNY',
 'RSL',
 'SEA',
 'SJ',
 'SKC',
 'TOR',
 'VAN']

In [229]:
stat_df['name'] = stat_df['Player']
stat_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,GWG,PKG/A,HmG,RdG,G/90min,SC%,name
0,Diego Rossi,LAFC,F,19,19,1698,14,4,71,34,1,1/1,11,3,0.74,19.7,Diego Rossi
1,Gyasi Zardes,CLB,F,21,20,1717,12,4,41,19,2,0/2,8,4,0.63,29.3,Gyasi Zardes
2,Raul Ruidiaz,SEA,F,17,17,1427,12,4,55,23,4,1/1,8,4,0.76,21.8,Raul Ruidiaz
3,Robert Beric,CHI,F,23,22,1931,12,1,62,29,1,1/2,7,5,0.56,19.4,Robert Beric
4,Jordan Morris,SEA,F,22,18,1724,10,8,39,18,3,0/0,8,2,0.52,25.6,Jordan Morris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,Sebastian Mendez,ORL,M,19,11,1087,0,0,17,3,0,0/0,0,0,0.00,0.0,Sebastian Mendez
669,Kelyn Rowe,SEA,M,16,10,896,0,0,17,4,0,0/0,0,0,0.00,0.0,Kelyn Rowe
670,Siem de Jong,CIN,M,15,8,793,0,0,18,5,0,0/1,0,0,0.00,0.0,Siem de Jong
671,Maxime Chanot,NYC,D,20,20,1712,0,0,19,4,0,0/0,0,0,0.00,0.0,Maxime Chanot


In [58]:
player_df = pd.read_csv("data/player_bio.csv")
stat_df = pd.read_csv("data/play_time.csv")

In [59]:
from unidecode import unidecode

player_df['first_name'] = player_df['first_name'].apply(unidecode)
player_df['last_name'] = player_df['last_name'].apply(unidecode)

player_df['Player'] = player_df['first_name'] +" "+ player_df['last_name']
player_df = player_df[['player', 'club', 'value', 'age','join_age','Player']]
player_df

,player,club,value,age,join_age,Player
0,K. Vermeer,Los Angeles FC,€800Th.,35.0,34.0,Kenneth Vermeer
1,P. Sisniega,Los Angeles FC,€500Th.,25.0,23.0,Pablo Sisniega
2,T. Romero,Los Angeles FC,€100Th.,20.0,20.0,Tomas Romero
3,E. Segura,Los Angeles FC,€2.50m,24.0,22.0,Eddie Segura
4,J. Murillo,Los Angeles FC,€1.20m,27.0,26.0,Jesus Murillo
...,...,...,...,...,...,...
723,T. Schmitt,Real Salt Lake City,€200Th.,23.0,21.0,Tate Schmitt
724,C. Garcia,Real Salt Lake City,€100Th.,18.0,17.0,Christopher Garcia
725,D. Martínez,Real Salt Lake City,€600Th.,23.0,22.0,Douglas Martinez
726,R. Rubin,Real Salt Lake City,€400Th.,25.0,24.0,Rubio Rubin


In [60]:
player_df['Player'][264] = 'Ismael Tajouri-Shradi'
player_df['Player'][257] = 'Juan Pablo Torres'
player_df['Player'][261] = 'Maximiliano Moralez'

/Users/solgaleo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/solgaleo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
stat_df = stat_df[['Player', 'Club', 'MINS', 'POS']]
stat_df['Player'] = stat_df['Player'].apply(unidecode)
stat_df

,Player,Club,MINS,POS
0,Diego Rossi,LAFC,1698,F
1,Gyasi Zardes,CLB,1717,F
2,Raul Ruidiaz,SEA,1427,F
3,Robert Beric,CHI,1931,F
4,Jordan Morris,SEA,1724,F
...,...,...,...,...
668,Sebastian Mendez,ORL,1087,M
669,Kelyn Rowe,SEA,896,M
670,Siem de Jong,CIN,793,M
671,Maxime Chanot,NYC,1712,D


In [62]:
df_raw = pd.merge(stat_df, player_df, on='Player', how='left')

In [63]:
df = df_raw.dropna(subset=['club'])

In [64]:
df.reset_index(drop=True)

,Player,Club,MINS,POS,player,club,value,age,join_age
0,Diego Rossi,LAFC,1698,F,D. Rossi,Los Angeles FC,€20.00m,23.0,19.0
1,Gyasi Zardes,CLB,1717,F,G. Zardes,Columbus Crew SC,€3.50m,29.0,26.0
2,Raul Ruidiaz,SEA,1427,F,R. Ruidíaz,Seattle Sounders FC,€7.00m,30.0,27.0
3,Robert Beric,CHI,1931,F,R. Beric,Chicago Fire FC,€3.00m,29.0,28.0
4,Chris Mueller,ORL,1477,F,C. Mueller,Orlando City SC,€3.00m,24.0,21.0
...,...,...,...,...,...,...,...,...,...
470,Marcelo Silva,RSL,1309,D,M. Silva,Real Salt Lake City,€400Th.,31.0,28.0
471,Boniek Garcia,HOU,819,M,B. García,Houston Dynamo FC,€200Th.,36.0,27.0
472,Sacha Kljestan,LA,769,M,S. Kljestan,Los Angeles Galaxy,€350Th.,35.0,34.0
473,Kelyn Rowe,SEA,896,M,K. Rowe,Seattle Sounders FC,€600Th.,29.0,29.0


In [65]:
nyc_df = df[df['Club']=='NYC'].reset_index(drop=True)
nyc_df

,Player,Club,MINS,POS,player,club,value,age,join_age
0,Valentin Castellanos,NYC,1341,F,V. Castellanos,New York City FC,€4.00m,22.0,20.0
1,Jesus Medina,NYC,1495,M,J. Medina,New York City FC,€3.00m,23.0,20.0
2,Anton Tinnerholm,NYC,2001,D,A. Tinnerholm,New York City FC,€2.50m,30.0,26.0
3,Ismael Tajouri-Shradi,NYC,670,F,I. Tajouri-ShradiI,New York City FC,€1.00m,26.0,23.0
4,Keaton Parks,NYC,1733,M,K. Parks,New York City FC,€3.00m,23.0,22.0
5,Alexander Callens,NYC,1934,D,A. Callens,New York City FC,€1.80m,28.0,24.0
6,Maximiliano Moralez,NYC,875,M,M. Moralez,New York City FC,€1.50m,34.0,29.0
7,Tony Rocha,NYC,51,M,T. Rocha,New York City FC,€450Th.,27.0,25.0
8,Gudmundur Thorarinsson,NYC,683,D,G. Thórarinsson,New York City FC,€800Th.,28.0,27.0
9,James Sands,NYC,1409,M,J. Sands,New York City FC,€2.50m,20.0,16.0


In [66]:
df.to_csv('data/combine.csv')

In [115]:
import re

def convert_currency(s):
    a = re.findall(r'[A-Za-z]+.?', s)
    if a:
        if re.findall(r'[A-Za-z]+.?', s)[0] == 'm':
            return float(re.findall(r'\d+', s)[0])*1000000*1.18
        elif re.findall(r'[A-Za-z]+.?', s)[0] == 'Th.':
            return float(re.findall(r'\d+', t)[0])*1000*1.18
    return None

In [116]:
df['usd'] = df['value'].apply(convert_currency)

/Users/solgaleo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [117]:
df

,Player,Club,MINS,POS,player,club,value,age,join_age,usd
0,Diego Rossi,LAFC,1698,F,D. Rossi,Los Angeles FC,€20.00m,23.0,19.0,23600000.0
1,Gyasi Zardes,CLB,1717,F,G. Zardes,Columbus Crew SC,€3.50m,29.0,26.0,3540000.0
2,Raul Ruidiaz,SEA,1427,F,R. Ruidíaz,Seattle Sounders FC,€7.00m,30.0,27.0,8260000.0
3,Robert Beric,CHI,1931,F,R. Beric,Chicago Fire FC,€3.00m,29.0,28.0,3540000.0
5,Chris Mueller,ORL,1477,F,C. Mueller,Orlando City SC,€3.00m,24.0,21.0,3540000.0
...,...,...,...,...,...,...,...,...,...,...
664,Marcelo Silva,RSL,1309,D,M. Silva,Real Salt Lake City,€400Th.,31.0,28.0,472000.0
665,Boniek Garcia,HOU,819,M,B. García,Houston Dynamo FC,€200Th.,36.0,27.0,472000.0
667,Sacha Kljestan,LA,769,M,S. Kljestan,Los Angeles Galaxy,€350Th.,35.0,34.0,472000.0
670,Kelyn Rowe,SEA,896,M,K. Rowe,Seattle Sounders FC,€600Th.,29.0,29.0,472000.0


In [89]:
float(re.findall(r'\d+', t)[0])*1000*1.18

472000.0

In [118]:
df.to_csv("data/usd.csv")